In [ ]:
#installing kaggle library
! pip install kaggle   #purpose is to use API in order to extract the data directly from kaggle site

upload your kaggle.json file


In [ ]:
# it is standard code configuring the path of kaggle.json file
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

importing twitter sentiment dataset


In [ ]:
#if the file is small we can download but if the dataset is huge then it is easy to fetch API from the site
#api to fetch the dataset from kaggle
!kaggle datasets download -d kazanova/sentiment140

sentiment140.zip: Skipping, found more recently modified local copy (use --force to force download)


extracting the compressed dataset

In [ ]:
from zipfile import ZipFile
dataset='/content/sentiment140.zip'

with ZipFile(dataset,'r') as zip:
  zip.extractall()
  print('the dataset is extracted')

the dataset is extracted


importing the dependencies


In [ ]:
import numpy as np
import pandas as pd
import re
from nltk.corpus import stopwords   #nltk=natural lang toolkit library
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer  #Tfidfvectorizer is used to convert textual data into numerical data
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

downloading stopwords

In [ ]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
#printing the stopwords in English
print(stopwords.words('english'))  #stopwords are something that doesnt add any meaning to ur textual data it has some meaning  but it dont have any inflential meaning
#stopwords in any language are those words that dont add that much importance to the context importance

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

data processing

In [ ]:
#loading data from csv file to pandas dataframe
twitter_data=pd.read_csv('training.1600000.processed.noemoticon.csv', encoding = 'ISO-8859-1')


In [ ]:
#CHECKING THE NO OF ROWS AND COLUMNS
twitter_data.shape  #0 to 1600000

(1599999, 6)

In [ ]:
twitter_data.dtypes

0                                                                                                                       int64
1467810369                                                                                                              int64
Mon Apr 06 22:19:45 PDT 2009                                                                                           object
NO_QUERY                                                                                                               object
_TheSpecialOne_                                                                                                        object
@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer.  You shoulda got David Carr of Third Day to do it. ;D    object
dtype: object

In [ ]:
#printing frst 5 rows of dataframe
twitter_data.head()

,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer. You shoulda got David Carr of Third Day to do it. ;D"
0,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
1,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
2,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
3,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
4,0,1467811372,Mon Apr 06 22:20:00 PDT 2009,NO_QUERY,joy_wolf,@Kwesidei not the whole crew


In [ ]:
# naming the columns and reading dataset again
column_names=['target','id','date','flag','user','text']
twitter_data=pd.read_csv('training.1600000.processed.noemoticon.csv',names=column_names, encoding = 'ISO-8859-1')

In [ ]:
#CHECKING THE NO OF ROWS AND COLUMNS
twitter_data.shape

(1600000, 6)

In [ ]:
#printing frst 5 rows of dataframe
twitter_data.head()

,target,id,date,flag,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [ ]:
twitter_data.dtypes

target     int64
id         int64
date      object
flag      object
user      object
text      object
dtype: object

In [ ]:
#counting the number of missing values in the datset
twitter_data.isnull().sum()   #this tells how many missing values are there

target    0
id        0
date      0
flag      0
user      0
text      0
dtype: int64

In [ ]:
#checking the distribution of target column
twitter_data['target'].value_counts() #valuecounts counts the number of values of 0 and 4

0    800000
4    800000
Name: target, dtype: int64

convert the target "4" to "1"

In [ ]:
twitter_data.replace({'target':{4:1}}, inplace=True) #inplace=True means the conversion has to be stored in my original dataframe
#if we dont place inplace then the conversion wont store in the dataframe

In [ ]:
#checking the distribution of target column
twitter_data['target'].value_counts()

0    800000
1    800000
Name: target, dtype: int64

"0" --->negative tweet

"1" ---> positive tweet

Stemming : it is the process of reducing a word to its root word
EX: actor,actress,acting = act
reason for doing this is :dataset is huge and we need to reduce the dimensions and complexity of the data
if we reduce the words to its root word then it will be easier for ml models for classifying whether positive or negative

In [ ]:
port_stem = PorterStemmer()

In [ ]:
# just a template of stemming operation
def stemming(content):   # passes a paremeter content is the text column from dataframe
  stemmed_content = re.sub('[^a-zA-Z]',' ',content)  #from regular expression re library import sub() #remove the letters which dont belong to the alphabets like spcl chars etc
  stemmed_content = stemmed_content.lower() # convert all letters to lowercase letter
  stemmed_content = stemmed_content.split()  #spliting the words from tweet and load it to a list
  stemmed_content = [port_stem.stem(word) for word in stemmed_content if not word in stopwords.words('english')]   #if a word dont belong to the stopwords then we perform the stemming  if the word is present in the stopwords then we dont perform stemming and just ignore those words
  stemmed_content = ' '.join(stemmed_content) #joining all the words in a particular tweet

  return stemmed_content

Applying this stemming data to the text data




In [ ]:
#creating a new column named stemmed_content the data is stemmed content
twitter_data['stemmed_content'] = twitter_data['text'].apply(stemming)  #stemming is the stemming operation function

In [ ]:
twitter_data.head()

,target,id,date,flag,user,text,stemmed_content
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t...",switchfoot http twitpic com zl awww bummer sho...
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...,upset updat facebook text might cri result sch...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...,kenichan dive mani time ball manag save rest g...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire,whole bodi feel itchi like fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all....",nationwideclass behav mad see


In [ ]:
print(twitter_data['stemmed_content'])

0          switchfoot http twitpic com zl awww bummer sho...
1          upset updat facebook text might cri result sch...
2          kenichan dive mani time ball manag save rest g...
3                            whole bodi feel itchi like fire
4                              nationwideclass behav mad see
                                 ...                        
1599995                           woke school best feel ever
1599996    thewdb com cool hear old walt interview http b...
1599997                         readi mojo makeov ask detail
1599998    happi th birthday boo alll time tupac amaru sh...
1599999    happi charitytuesday thenspcc sparkschar speak...
Name: stemmed_content, Length: 1600000, dtype: object


In [ ]:
print(twitter_data['target'])

0          0
1          0
2          0
3          0
4          0
          ..
1599995    1
1599996    1
1599997    1
1599998    1
1599999    1
Name: target, Length: 1600000, dtype: int64


In [ ]:
#seperating the data and label.Here data is the tweet and label is the target i.e 0 or 1
X = twitter_data['stemmed_content'].values
Y = twitter_data['target'].values

In [ ]:
print(X)

['switchfoot http twitpic com zl awww bummer shoulda got david carr third day'
 'upset updat facebook text might cri result school today also blah'
 'kenichan dive mani time ball manag save rest go bound' ...
 'readi mojo makeov ask detail'
 'happi th birthday boo alll time tupac amaru shakur'
 'happi charitytuesday thenspcc sparkschar speakinguph h']


In [ ]:
print(Y)

[0 0 0 ... 1 1 1]


Splitting the data to training data and test data

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X,Y,test_size=0.2,shuffle=True, stratify=Y,random_state=42) #test_size is how many datapoints you need for test data here we use 20% of my data will go to testdata and 80% will go to train data
#stratify means in training and test data the proportion of 0 and 1 should be same

In [ ]:
print(X.shape, X_train.shape,X_test.shape)

(1600000,) (1280000,) (320000,)


In [ ]:
print(X_train)

['paisleypaisley lol get idea far advanc even june yet need third knitter summer group'
 'worst headach ever'
 'ewaniesciuszko sad wont see miss alreadi yeah perfect come back th' ...
 'got home meet talk endlessli one coolest guy ever met smile'
 'bought chocol bar quot win free bar quot label win either'
 'misecia said hope dm email sunday']


In [ ]:
print(X_test)

['stm denali ye black red fav color realli want color def look awesom jare'
 'qu buy open hous weekend pm best valu one bedroom lic long island citi bd http tinyurl com pt nqd'
 'ginoandfran fran greet air okay hahahaha thank' ...
 'la brat follow also hope atleast get also wish get well soon'
 'feel like decent swell sinc last fall hope wave myrtl beach week either least golf'
 'relaxin busi day']


In [ ]:
print(Y.shape, Y_train.shape,Y_test.shape)

(1600000,) (1280000,) (320000,)


Feature Extraction : converting the textual data to numerical data

In [ ]:
vectorizer = TfidfVectorizer()
# this vectorizer will consider each word and it will give some importance to each word this importance of data done based on the repetition of the word and correspond to value either 0 or 1
X_train = vectorizer.fit_transform(X_train) #we should fit and transform the training data .Here fitting tries to understand the nature of the data that is present and transform convert the data to the textual data to numerical values
X_test = vectorizer.transform(X_test) #here we only transform the test data based on training data the test data will transsform

In [ ]:
print(X_train) #here 0 means the words related to first tweet and so on

  (0, 154767)	0.2697660704325824
  (0, 388138)	0.20555120011808467
  (0, 220296)	0.4301567790762487
  (0, 406297)	0.2978221095272138
  (0, 286478)	0.16123218610004275
  (0, 454381)	0.20169626473577718
  (0, 205794)	0.2414022906380175
  (0, 124524)	0.18318401951949756
  (0, 4832)	0.31707426786115905
  (0, 128605)	0.22108856600702775
  (0, 175252)	0.22407080547034602
  (0, 146067)	0.12929728405657018
  (0, 239679)	0.15130037108228483
  (0, 307108)	0.4620604881532448
  (1, 124611)	0.5113765148324884
  (1, 161801)	0.5778049407933611
  (1, 445870)	0.6361096685891185
  (2, 400192)	0.2722541116865256
  (2, 31063)	0.1936303169258752
  (2, 78861)	0.21039643874061958
  (2, 312657)	0.3154702974657607
  (2, 453420)	0.2347069337186747
  (2, 12436)	0.2529872032123258
  (2, 267649)	0.19309660201644555
  (2, 358186)	0.19837942712286838
  :	:
  (1279997, 120136)	0.5183864288132355
  (1279997, 80839)	0.4085075096529922
  (1279997, 262076)	0.3257051331036561
  (1279997, 372797)	0.3021907911926725
  (1279

In [ ]:
print(X_test)

  (0, 453357)	0.1781708363247895
  (0, 435956)	0.14183025329879742
  (0, 384697)	0.3281164007446601
  (0, 335577)	0.22602158147814247
  (0, 334643)	0.14719329779308424
  (0, 240451)	0.15341308097014625
  (0, 189057)	0.31324918577405797
  (0, 129417)	0.25650960779862714
  (0, 97585)	0.4019235611854435
  (0, 96399)	0.255967788489452
  (0, 78636)	0.5158100011206617
  (0, 43712)	0.23562815302828183
  (0, 28874)	0.1778395103911245
  (1, 438137)	0.16390196841168053
  (1, 428475)	0.2850923429146983
  (1, 409742)	0.19539196331452124
  (1, 328541)	0.30370566878650734
  (1, 325414)	0.2832875855026037
  (1, 318909)	0.20246938925608493
  (1, 303003)	0.19949441578138133
  (1, 301683)	0.13212235134015302
  (1, 240223)	0.1674195650536303
  (1, 233854)	0.3852709938491561
  (1, 183279)	0.24586158827112847
  (1, 171245)	0.12468774856570086
  :	:
  (319997, 225096)	0.2933179603743362
  (319997, 169469)	0.19450444826456267
  (319997, 146067)	0.3148589224636577
  (319997, 135536)	0.2180996497753238
  (3199

Training the ml model

Logistic Regression

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
lr_model = LogisticRegression(max_iter=1000) #max number of times the model can go through the data


In [ ]:
lr_model.fit(X_train,Y_train) #fit() is where the model from the data

LogisticRegression(max_iter=1000)

Model Evaluation

Accuracy score on the training data

In [ ]:
X_train_predict = lr_model.predict(X_train)
training_data_accuracy = accuracy_score(Y_train,X_train_predict)

In [ ]:
print('Accuracy on training data',training_data_accuracy)

Accuracy on training data 0.81021171875


Accuracy score on the test data

In [ ]:
X_test_predict = lr_model.predict(X_test)
test_data_accuracy = accuracy_score(Y_test,X_test_predict)

In [ ]:
print('Accuracy on test data',test_data_accuracy)

Accuracy on test data 0.778690625


In [ ]:
from sklearn.svm import SVC
# Support Vector Machine (SVM) model
svm_model = SVC(kernel='linear',random_state = 42)  # You can experiment with different kernels
svm_model.fit(X_train, Y_train)

Saving the trained model

In [ ]:
import pickle

In [ ]:
filename='sent_trainedmodel.sav'
pickle.dump(model, open(filename,'wb'))

In [ ]:
import sklearn
sklearn_version = sklearn.__version__
print(sklearn_version)

Using saved model for future predictions

In [ ]:
#loading saved models
loaded_model = pickle.load(open('/content/sent_trainedmodel.sav','rb'))

In [ ]:
X_new = X_test[200]
print(Y_test[200])

prediction = model.predict(X_new) #result of prediction will be stored in list
print(prediction)
if(prediction[0] == 0):
  print('negative tweet')

else:
  print('positive tweet')

In [ ]:
X_new = X_test[2]
print(Y_test[2])

prediction = model.predict(X_new) #result of prediction will be stored in list
print(prediction)
if(prediction[0] == 0):
  print('negative tweet')

else:
  print('positive tweet')

Model Deployment

In [ ]:
import pickle
import numpy as np
import streamlit as st
model = pickle.load(open('/content/sent_trainedmodel.sav','rb'))
st.title('Sentiment Analysis Model')

st.write('Enter Text for Sentiment analysis:')
message = st.text_area("","Type Here ....")

array = np.fromstring(message, dtype=float, sep=' ')
reshaped_array = array.reshape(-1, 1)
if st.button('PREDICT'):
  disp=" "
  prediction = model.predict([reshaped_array])[0] #result of prediction will be stored in list
  print(prediction)
  if(prediction[0] == 0):
    disp='negative tweet'

  else:
    disp='positive tweet'
  st.write('the given text is:',disp)